# گزارش آماری و تحلیلی (post-processing)

In [1]:
!pip install xgboost pandas numpy scipy joblib matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 39.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install scikit-learn==1.6.1 xgboost==2.0.3 pandas==2.2.2 numpy==1.26.4 scipy==1.14.1 joblib==1.4.2 matplotlib==3.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 943.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.2
    Uninstalling joblib-1.5.2:
      Successfully uninstalled joblib-1.5.2
  Attempting uninstall: xgboost
    Found existing installation: xgboost 3.1.1
    Uninstalling xgboost-3.1.1:
      Successfully uninstalled x

In [6]:
# نصب نسخه‌های سازگار (برای رفع خطای ColumnTransformer)
!pip install scikit-learn==1.6.1 xgboost==2.0.3 pandas==2.2.2 numpy==1.26.4 scipy==1.14.1 joblib==1.4.2 matplotlib==3.9.2 seaborn==0.13.2

import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import os

# چک کردن فایل‌ها
print("فایل‌های موجود در root:")
print(os.listdir('.'))

# مسیرها (فایل‌ها در root پروژه هستند)
DATA_PATH = "id_columns_step11.csv"
MODEL_PATH = "best_xgb_model1.json"
PREPROCESSOR_PATH = "preprocessor.pkl"

# معانی کامل تمام ستون‌ها (حتی اگر در داده نباشند)
meanings = {
    'noe_tasadof_id': {0: 'جرحی', 1: 'خسارتی', 2: 'فوتی'},
    'noe_barkhord_id': {0: 'موتورسیکلت با وسیله نقلیه', 1: 'وسیله نقلیه با یک وسیله نقلیه', 2: 'وسیله نقلیه با عابر',
                        3: 'موتورسیکلت با عابر', 4: 'وسیله نقلیه با شیء ثابت', 5: 'موتورسیکلت - واژگونی',
                        6: 'وسیله نقلیه - واژگونی', 7: 'موتورسیکلت با شیء ثابت', 8: 'موتور با وسیله نقلیه پارک شده',
                        9: 'وسیله نقلیه - خروج از جاده', 10: 'سایر', 11: 'وسیله نقلیه با دوچرخه', 12: 'پرتاب سرنشین',
                        13: 'موتورسیکلت با دوچرخه', 14: 'نامشخص', 15: 'موتورسیکلت - خروج از جاده',
                        16: 'موتورسیکلت با موتورسیکلت', 17: 'یک وسیله نقلیه', 18: 'چند وسیله نقلیه',
                        19: 'وسیله نقلیه با موتور سیکلت', 20: 'دوچرخه با موتورسیکلت', 21: 'مینی بوس به سواری',
                        22: 'موتورسیکلت با چند وسیله دیگر', 23: 'وسیله نقلیه پارک شده',
                        24: 'وسیله نقلیه با چند وسیله نقلیه و عابر', 25: 'موتورسیکلت', 26: 'واژگونی و سقوط',
                        27: 'وسیله نقلیه با موتورسیکلت', 28: 'وسیله نقلیه با حیوان/احشام',
                        29: 'واژگونی و سقوط، خروج از جاده', 30: 'کامیون با وسیله نقلیه',
                        31: 'وسیله نقلیه با چند وسیله نقلیه - عابر', 32: 'وسیله نقلیه با دوچرخه و چند وسیله',
                        33: 'موتورسیکلت با حیوان/احشام', 34: 'چند برخوردی'},
    'vaze_roshanaei_id': {0: 'روز', 1: 'شب با روشنایی کافی', 2: 'غروب', 3: 'شب بدون روشنایی کافی',
                          4: 'نامشخص', 5: 'طلوع', 6: 'سایر', 7: 'شب با روشنایی نامشخص'},
    'sharayet_sathe_rah_id': {0: 'خشک و معمولی', 1: 'مرطوب و خیس', 2: 'یخبندان و برفی', 3: 'نامشخص',
                              4: 'سایر', 5: 'شنی و خاکی', 6: 'گل آلود'},
    'noe_shane_id': {0: 'شانه ندارد', 1: 'شانه خاکی', 2: 'شانه آسفالته', 3: 'نامشخص'},
    'vaze_hava_id': {0: 'صاف', 1: 'ابری', 2: 'بارانی', 3: 'برفی', 4: 'نامشخص', 5: 'بارانی و ابری',
                     6: 'مه آلود', 7: 'غبارآلود', 8: 'صاف - ابری'},
    'khatkeshi_id': {0: 'ندارد', 1: 'منقطع', 2: 'ممتد', 3: 'دوبل', 4: 'نامشخص', 5: 'ممتد و دوبل', 6: 'ممتد و منقطع'},
}

# بارگذاری و پاکسازی داده
try:
    df = pd.read_csv(DATA_PATH)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.drop_duplicates()
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=['noe_tasadof_id'])
    print(f"داده با موفقیت بارگذاری شد: {df.shape[0]} ردیف، {df.shape[1]} ستون")
except Exception as e:
    print(f"خطا در بارگذاری داده: {e}")
    raise

# ویژگی‌ها
features = ['khatkeshi_id', 'noe_barkhord_id', 'vaze_roshanaei_id', 'sharayet_sathe_rah_id',
            'noe_shane_id', 'vaze_hava_id', 'mavane_did_id', 'naqayes_moaser_rah_id']

# شروع گزارش
report = "گزارش جامع آماری تحلیل شدت تصادفات جاده‌ای\n"
report += "="*70 + "\n\n"
report += f"تعداد کل تصادفات تحلیل‌شده: {len(df)}\n\n"
report += "توزیع کلی شدت تصادف:\n"
dist = df['noe_tasadof_id'].value_counts(normalize=True) * 100
for idx in sorted(dist.index):
    report += f"  • {meanings['noe_tasadof_id'][idx]}: {dist[idx]:.1f}%\n"
report += "\n"

# تحلیل جامع هر ویژگی (با نمایش تمام حالت‌ها حتی اگر صفر باشند)
for feature in features:
    report += f"\n{'='*80}\n"
    report += f"تحلیل کامل ویژگی: {feature}\n"
    report += f"{'='*80}\n\n"
    
    if feature in meanings:
        # ایجاد ستون معنی
        df[feature + '_label'] = df[feature].map(meanings[feature])
        
        # همه دسته‌های ممکن
        all_labels = list(meanings[feature].values())
        label_to_code = {v: k for k, v in meanings[feature].items()}
        
        # شمارش واقعی
        count_table = pd.crosstab(df[feature + '_label'], df['noe_tasadof_id'], margins=False)
        
        # اضافه کردن دسته‌های غایب
        for label in all_labels:
            if label not in count_table.index:
                count_table.loc[label] = 0
        
        # مرتب‌سازی بر اساس کد اصلی (0,1,2,...)
        count_table = count_table.reindex(all_labels, fill_value=0)
        
        # تبدیل به درصد
        percent_table = count_table.div(count_table.sum(axis=1), axis=0) * 100
        percent_table = percent_table.round(1).fillna(0)
        percent_table.columns = [meanings['noe_tasadof_id'][c] for c in percent_table.columns]
        
        # ذخیره در گزارش
        report += f"توزیع شرطی شدت تصادف بر اساس {feature}:\n\n"
        report += percent_table.to_string() + "\n\n"
        report += "توضیح: مقادیر 0.0 نشان‌دهنده عدم وجود آن حالت در داده است.\n\n"
        
        # تست Chi-square
        chi2_table = pd.crosstab(df[feature], df['noe_tasadof_id'])
        chi2, p, _, _ = chi2_contingency(chi2_table)
        sig = "معنادار (وابستگی قوی)" if p < 0.05 else "غیرمعنادار"
        report += f"تست خی‌دو (Chi-square): χ² = {chi2:.2f}, p-value = {p:.4f} → {sig}\n\n"
        
        # تولید Heatmap با تمام دسته‌ها
        plt.figure(figsize=(10, max(6, len(all_labels) * 0.35)))
        sns.heatmap(percent_table, annot=True, fmt='.1f', cmap='Reds', cbar_kws={'label': 'درصد (%)'})
        plt.title(f'توزیع شدت تصادف بر اساس {feature}\n(تمام حالت‌های ممکن نمایش داده شده)', fontsize=14, pad=20)
        plt.xlabel('شدت تصادف', fontsize=12)
        plt.ylabel('مقدار ویژگی', fontsize=12)
        plt.tight_layout()
        plt.savefig(f'heatmap_full_{feature}.png', dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Heatmap کامل ذخیره شد: heatmap_full_{feature}.png")
        
    else:
        # برای mavane_did_id و naqayes_moaser_rah_id
        observed = df[feature].dropna().unique()
        count_table = pd.crosstab(df[feature], df['noe_tasadof_id']).reindex(observed, fill_value=0)
        percent_table = count_table.div(count_table.sum(axis=1), axis=0) * 100
        percent_table = percent_table.round(1).fillna(0)
        percent_table.columns = [meanings['noe_tasadof_id'][c] for c in percent_table.columns]
        
        report += f"توزیع شدت تصادف برای {feature} (فقط مقادیر موجود):\n\n"
        report += percent_table.to_string() + "\n\n"
        
        chi2, p, _, _ = chi2_contingency(count_table)
        sig = "معنادار" if p < 0.05 else "غیرمعنادار"
        report += f"Chi-square: p-value = {p:.4f} → {sig}\n\n"
        
        plt.figure(figsize=(10, 6))
        sns.heatmap(percent_table, annot=True, fmt='.1f', cmap='Blues')
        plt.title(f'توزیع شدت تصادف بر اساس {feature}')
        plt.xlabel('شدت تصادف')
        plt.ylabel(feature)
        plt.tight_layout()
        plt.savefig(f'heatmap_full_{feature}.png', dpi=300, bbox_inches='tight')
        plt.close()

# بخش مدل XGBoost (با مدیریت خطا)
report += "\n\n" + "="*80 + "\n"
report += "اهمیت ویژگی‌ها از مدل XGBoost\n"
report += "="*80 + "\n"

try:
    preprocessor = joblib.load(PREPROCESSOR_PATH)
    model = xgb.XGBClassifier()
    model.load_model(MODEL_PATH)
    
    X = df.drop('noe_tasadof_id', axis=1)
    X_processed = preprocessor.transform(X)
    importances = model.feature_importances_
    names = preprocessor.get_feature_names_out()
    
    imp_df = pd.DataFrame({'ویژگی': names, 'اهمیت': importances})
    imp_df = imp_df.sort_values('اهمیت', ascending=False)
    
    report += "بارگذاری مدل با موفقیت انجام شد.\n\n"
    report += "۱۰ ویژگی برتر:\n"
    report += imp_df.head(10).round(4).to_string(index=False) + "\n\n"
    
    plt.figure(figsize=(10, 7))
    top10 = imp_df.head(10)
    plt.barh(range(len(top10)), top10['اهمیت'])
    plt.yticks(range(len(top10)), top10['ویژگی'])
    plt.xlabel('اهمیت')
    plt.title('۱۰ ویژگی برتر بر اساس مدل XGBoost')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("نمودار اهمیت ویژگی‌ها ذخیره شد: feature_importance.png")

except Exception as e:
    report += f"خطا در بارگذاری مدل (احتمالاً ناسازگاری نسخه scikit-learn):\n{e}\n"
    report += "این بخش قابل اجرا نبود. تحلیل آماری و Heatmap‌ها همچنان معتبر هستند.\n"
    print(f"خطا در مدل: {e}")

# ذخیره نهایی گزارش
with open('گزارش_جامع_تحلیل_شدت_تصادفات.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n" + "="*70)
print("تحلیل کامل با موفقیت انجام شد!")
print("فایل گزارش متنی: گزارش_جامع_تحلیل_شدت_تصادفات.txt")
print("تمام Heatmap‌ها با پیشوند heatmap_full_ ذخیره شدند")
print("تمام حالت‌های ممکن در هر ویژگی نمایش داده شد")
print("="*70)
print(report)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 123.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.2
    Uninstalling joblib-1.5.2:
      Successfully uninstalled joblib-1.5.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successf

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e269dd1d-9489-4b22-a20c-1489fea40d6b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>